In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Numéro 1

The electronic density $n_e$ of a compound of $M$ elements is defined from its density $\rho$ and its composition of atoms of atomic numbers $N_i$ present with massic fractions of $f_i$ such that:
$$
    n_e = \rho\sum_{n=1}^MN_if_i
$$

In [4]:
def electronic_density(density, composition):
    """
    Calculate the electronic density of a compound.
    Params:
    density (float): The density of the compound in g/cm^3.
    composition (dict): A dictionary with the elements' atomic number as keys and the fraction (on 1) by weight of that element as values.
    Returns:
    float: The electronic density of the compound in g/cm^3.
    """
    electronic_density = 0
    for element in composition:
        electronic_density += element * composition[element]
    electronic_density *= density
    return electronic_density

In [5]:
# from NIST for liquid water (https://physics.nist.gov/cgi-bin/Star/compos.pl?refer=ap&matno=276)
liquid_water_density = 1
liquid_water_composition = {1: 0.111894, 8: 0.888106}
liquid_water_ne = electronic_density(liquid_water_density, liquid_water_composition)
print(f"The electronic density of liquid water is {liquid_water_ne:.2f} g/cm^3.")

# from NIST for compact bone (https://physics.nist.gov/cgi-bin/Star/compos.pl?refer=ap&matno=119)
compact_bone_density = 1.85
compact_bone_composition = {1: 0.063984, 6: 0.278000, 7: 0.027000, 8: 0.410016, 12: 0.002000, 15: 0.070000, 16: 0.002000, 20: 0.147000}
compact_bone_ne = electronic_density(compact_bone_density, compact_bone_composition)
print(f"The electronic density of compact bone is {compact_bone_ne:.2f} g/cm^3.")

The electronic density of liquid water is 7.22 g/cm^3.
The electronic density of compact bone is 17.11 g/cm^3.


## Numéro 2

The massic stopping power $S_{col, \rho}$ is defined like:
$$
\begin{aligned}
    S_{col, \rho} = \frac{S_{col}}{\rho} &= \frac{2\pi r_e^2m_ec^2n_eZ^2}{\rho\beta^2}\left[\ln\left(\frac{2m_ec^2(\gamma^2-1)T_e^{max}}{I^2}\right)-2\beta^2-\delta-2\frac{C}{Z}+2ZL_1+2Z^2L_2\right]\\
    &= \frac{2\pi r_e^2m_ec^2n_eZ^2}{\rho\beta^2}\left[\ln\left(\frac{2m_ec^2\beta^2\left(\frac{T}{m_pc^2}+1\right)^2T_e^{max}}{I^2}\right)-2\beta^2-\delta-2\frac{C}{Z}+2ZL_1+2Z^2L_2\right]\\
\text{where } T_e^{max} \text{ is defined and simplified as:}\\
    T_e^{max} &= \frac{2m_ec^2(\gamma^2-1)}{1+2\gamma\left(\frac{m_e}{m_p}\right)+\left(\frac{m_e}{m_p}\right)^2} = \frac{2m_ec^2\beta^2\left(\frac{T}{m_pc^2}+1\right)^2}{1+2\left(\frac{T}{m_pc^2}+1\right)\left(\frac{m_e}{m_p}\right)+\left(\frac{m_e}{m_p}\right)^2}
\end{aligned}
$$

In [3]:
def massic_stopping_power(n_e, m_e, Z, beta, rho, T, I, m_p, delta, C, L1, L2):
    """
    Calculate the massic stopping power of a material.
    Params:
    n_e (float): The electronic density of the material in g/cm^3.
    m_e (float): The electron mass in g.
    Z (float): The atomic number of the material.
    beta (float): The velocity of the incident particle as a fraction of the speed of light.
    rho (float): The density of the material in g/cm^3.
    Returns:
    float: The massic stopping power of the material in MeV cm^2/g.
    """
    c = 3e10 # speed of light in cm/s 
    r_e = 2.8179e-13 # radius of the electron in cm
    T_e_max = 2 * m_e * c**2 * beta**2*((T / (m_p * c**2)) + 1)**2 / (1 + 2 * (T / (m_p * c**2)) + 1 * (m_e / m_p) + (m_e / m_p)**2)
    prefactor = (2 * np.pi * r_e**2 * m_e * c**2 * n_e * Z**2) / (rho * beta**2)
    log_argument = (2 * m_e * c**2 * beta**2 * ((T / (m_p * c**2)) + 1)**2 * T_e_max) / I**2
    logarithmic_term = np.log(log_argument) - 2 * beta**2 - delta - 2 * (C / Z) + 2 * Z * L1 + 2 * Z**2 * L2
    return prefactor * logarithmic_term

In [ ]:
liquid_water_Z 

## Numéro 3

Lorsqu’un proton entre en collision avec la matière, deux principaux phénomènes entraînent sa décélération. Ces phénomènes sont les interactions nucléaires et les interactions radiatives.Les interactions nucléaires correspondent aux interactions entre les protons incidents et les noyaux présents dans la matière. Quant aux interactions radiatives, elles se produisent entre les protons incidents et les électrons du milieu.

Le NIST fournit une base de données identifiant l’ampleur de ces interactions individuelles. Voici ces interactions représentées dans un même graphique.

In [19]:
def read_pstar_data(filename):
    """
    Reads a text file and saves the data into a NumPy array.
    
    Parameters:
        filename (str): Path to the text file.
    
    Returns:
        np.ndarray: NumPy array containing the data.
    """
    data = np.loadtxt(filename, delimiter=" ", dtype=float, skiprows=8)
    try:
        data = np.loadtxt(filename, delimiter=" ", dtype=float, skiprows=8)
        return data
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    
class Medium:
    def __init__(self, data):
        self.data = data
        print(data)
        self.nuclear_stoping_power = self.get_nucleare_stoping_power()
        self.electronic_stoping_power = self.get_electronic_stoping_power()

    def get_electronic_stoping_power(self):
        
        return np.concatenate((self.data[:, 0], self.data[:, 1]))
    
    def get_nucleare_stoping_power(self):
        return np.concatenate((self.data[:, 0], self.data[:, 2]))
    
    def graph_all_stoping_power(self):
        plt.plot(self.nuclear_stoping_power[:,0], self.nuclear_stoping_power[:,1], label="Nuclear stopping power")
        plt.plot(self.electronic_stoping_power[:,0], self.electronic_stoping_power[:,1], label="Electronic stopping power")
        plt.xlabel("Energy (MeV)")
        plt.ylabel("Stopping power (MeV cm^2/g)")
        plt.legend()
        plt.show()
        

In [ ]:
read_pstar_data("tp1/waterLiquidStoppingPower.txt")
#water = Medium(read_pstar_data("tp1/waterLiquidStoppingPower.txt"))
#water.graph_all_stoping_power()

FileNotFoundError: ./tp1/waterLiquidStoppingPower.txt not found.

## Numéro 4

Plus le pouvoir d'arrêt d'un matériel est grand, plus la portée de la particule sera faible dans le matériel. Il est donc logique et intuitif de dire que la portée suit une tendance inverse au pouvoir d'arrêt. 

Il est également possible de démontrer cela en utilisant les unités. Nous cherchons une portée (unités: cm, dimension: $L$) à partir du pouvoir d'arrêt (unités: MeV/cm, dimensions: $ML/T^2$) et de l'énergie (unités: MeV, dimensions: $ML^2/T^2$). Notre seul moyen d'obtenir la dimension adéquate pour la portée est de multiplier les dimensions de l'énergie par les dimensions de l'inverse du pouvoir d'arrêt : $ML^2/T^2 \cdot (ML/T^2)^{-1} = L$.

C'est exactement ce que nous retrouvons pour la formule de la portée CSDA: $R_{CSDA} = \int_0^{T_i} \frac{1}{S} dT$

## Numéro 5

La formule pour le pouvoir d'arrêt est relativement complexe, et nous retrouvons avec une intégrale de forme $\int \frac{1}{ln(T)+B} dT$, où B est une constante. Cette intégrale ne produit pas de solution analytique, et donc l'utilisation d'une méthode analytique est nécessaire.  

## Numéro 6

## Numéro 7

## Numéro 8

## Numéro 9

## Numéro 10

## Numéro 11

## Numéro 12

## Numéro 13

## Numéro 14

## Numéro 15

## Numéro 16

## Numéro 17